# Track data lineage

{class}`~lamindb.File` objects are the {attr}`~lamindb.Run.inputs` and {attr}`~lamindb.Run.outputs` of run {class}`~lamindb.Run` objects. What is run is a {class}`~lamindb.Transform`.

From a `File` object, access the generating {class}`~lamindb.Run` and {class}`~lamindb.Transform` via {attr}`~lamindb.File.run` and {attr}`~lamindb.File.transform`.

In [ ]:
# initialize a test instance for this notebook
!lamin delete myobjects
!lamin init --storage ./myobjects

In [ ]:
import lamindb as ln

## Notebooks

Let us first track the data source. Here, it's a Jupyter notebook, so we can run:

In [ ]:
ln.track()

:::{dropdown} Track a pipeline instead of a notebook

If this is run in a pipeline, we need to pass a {class}`~lamindb.Transform` object of `type` "pipeline":

```
transform = ln.Transform("My script")  # optionally pass type="pipeline"
ln.track(transform)
```

This readily creates a {class}`~lamindb.Run` for the pipeline.

A pipeline is any non-interactive session: any Python script or workflow tool you may use.

:::

A global run context is created upon `ln.track()`:

In [ ]:
ln.context.transform

In [ ]:
ln.context.run

Let's add a file:

In [ ]:
filepath = ln.dev.datasets.file_mini_csv()
filepath = filepath.rename(ln.setup.settings.storage.root / filepath.name)

In [ ]:
file = ln.File(filepath)
ln.save(file)

We see that now, the file has transform and run records that are not `None`!

In [ ]:
file.transform

And hence, we can query for it!

In [ ]:
ln.select(ln.File, transform=ln.context.transform).one()

## Pipelines

In [ ]:
filepath = ln.dev.datasets.file_fastq()

When working with a pipeline, we'll register it before running it.

In [ ]:
transform = ln.Transform(
    name="10x scRNA-seq nextseq", type="pipeline"
)  # `type` defaults to `"pipeline"` outside of an ipython environment

We can then use the {func}`~lamindb.track` as before:

In [ ]:
ln.track(transform)

In [ ]:
file_fastq = ln.File(filepath)

In [ ]:
ln.save(file_fastq)

:::{dropdown}

We can also manually pass a run and not use the global run context set by `ln.track`:
```
run = ln.Run(transform=transform, name="ingest-fastq")
ln.File(filepath, run=run)
```

:::

## Track run inputs

While run outputs are automatically tracked as data sources, run inputs aren't.

Let's register a pipeline that takes the fastq file as an input:

In [ ]:
ln.track(ln.Transform(name="Cell Ranger", version="7", type="pipeline"))

To track it as an input for the current run, set `is_run_input=True`.

In [ ]:
file_fastq = ln.select(ln.File, name="input.fastq.gz").one()
file_fastq.stage(is_run_input=True)

Let's get an exemplary output filepath:

In [ ]:
output_filepath = ln.dev.datasets.file_bam()

In [ ]:
output_filepath

In [ ]:
file = ln.File(output_filepath)

ln.save(file)

Let's query the input file of the run that produced file `output.bam`:

In [ ]:
file = ln.select(ln.File, name="output.bam").one()
print(file.run.inputs.all())
print(file.run.outputs.all())

In [ ]:
assert file.run.inputs.all()[0].name == "input.fastq.gz"

## Query by provenance

Which notebook ingested a given file?

In [ ]:
file = ln.select(ln.File, name="mini.csv").first()
print(file.transform)

Which transforms were created by testuser2?

In [ ]:
ln.select(ln.Transform, created_by__handle="testuser1").df()

Which notebooks were created by testuser2?

In [ ]:
ln.select(ln.Transform, created_by__handle="testuser1", type="notebook").df()

### Users

Which users have interacted with the database?

In [ ]:
ln.select(ln.User).df()

Which user ingested this file `input.fastq.gz`?

In [ ]:
ln.select(ln.File, name="input.fastq.gz").one().created_by

Which users created notebooks with "lineage" in the title?

In [ ]:
ln.select(ln.Transform, name__contains="provenance").df()

In [ ]:
!lamin delete myobjects